In [1]:
!nvidia-smi

Tue Sep 27 19:15:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth

## Install Requirements

In [ ]:
!git clone https://github.com/ShivamShrirao/diffusers
!pip install git+https://github.com/ShivamShrirao/diffusers             # 2 times for now cause colab wasn't able to find it for some reason. Will fix later prolly.
%pip install -U --pre triton
%pip install accelerate
%pip install transformers
%pip install ftfy
%pip install bitsandbytes

### Install xformers from precompiled wheel. (These were compiled on Tesla T4, thanks to https://github.com/metrolobo)

In [ ]:
%pip install https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl

# if precompiled wheels don't work, install it with the following command. It will take around 40 minutes to compile.
# %pip install git+https://github.com/facebookresearch/xformers@1d31a3a#egg=xformers

## Login to HF and run

In [3]:
%cd /content/diffusers/examples/dreambooth

/content/diffusers/examples/dreambooth


In [4]:
from huggingface_hub import notebook_login
!git config --global credential.helper store
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [5]:
CLASS_NAME = "guy"          # just a general name for class like dog for dog images.

In [6]:
MODEL_NAME = "CompVis/stable-diffusion-v1-4"
INSTANCE_DIR = "/content/data/sks"
!mkdir -p $INSTANCE_DIR
CLASS_DIR = f"/content/data/{CLASS_NAME}"
OUTPUT_DIR = "/content/models/sks"            # sks is a rare identifier, feel free to replace it.

In [ ]:
# Upload your images by running this cell OR you can use the file manager on the left panel to upload to INSTANCE_DIR
import os
from google.colab import files
import shutil

uploaded = files.upload()
for filename in uploaded.keys():
    dst_path = os.path.join(INSTANCE_DIR, filename)
    print(f'move {filename} to {dst_path}')
    shutil.move(filename, dst_path)

In [9]:
# remove --use_8bit_adam flag if you got more than 18 GB VRAM.
!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME --use_auth_token \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="photo of sks {CLASS_NAME}" \
  --class_prompt="photo of a {CLASS_NAME}" \
  --seed=1337 \
  --resolution=512 \
  --center_crop \
  --train_batch_size=1 \
  --mixed_precision="no" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=12 \
  --sample_batch_size=4 \
  --max_train_steps=800


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--num_cpu_threads_per_process` was set to `1` to improve out-of-box performance
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
Moving 0 files to the new cache system
0it [00:00, ?it/s]

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
For effortless bug reporting copy-paste your error into this form: https://docs.goog

## Inference

In [20]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
from IPython.display import display

In [12]:
pipe = StableDiffusionPipeline.from_pretrained(OUTPUT_DIR, torch_dtype=torch.float16).to("cuda")

In [ ]:
g_cuda = torch.Generator(device='cuda')
# g_cuda.manual_seed(1337)

In [ ]:
num_samples = 4
prompt = ["photo of sks guy, digital painting"] * num_samples

with autocast("cuda"), torch.no_grad():
    images = pipe(prompt, height=512, width=512, num_inference_steps=50, guidance_scale=7.5, generator=g_cuda).images

for img in images:
    display(img)

  0%|          | 0/51 [00:00<?, ?it/s]